In [ ]:
import pandas as pd
import pyam
import nomenclature
import ixmp4

In [ ]:
iamc_args = dict(
    model="State of CDR (2024) - Current CDR",
    scenario="Observed Data",
)

In [ ]:
dsd = nomenclature.DataStructureDefinition("../definitions/")

In [ ]:
data = pd.read_excel(
    "source/7_Deployment/Indicator 15_Novel CDR volumes by method, year, and country.xlsx.xlsx",
    sheet_name="Dashboard",
    header=28,
    nrows=8,
    usecols="B:J",
).set_index("Year")

In [ ]:
data

In [ ]:
df = pyam.IamDataFrame(
    data,
    **iamc_args,
    value=data.columns,
    unit="t CO2",
    region="World",
)

In [ ]:
df

In [ ]:
df.variable

In [ ]:
df.rename(
    variable={
        "BECCS": "Bioenergy with CCS",
        "Bio-oil Storage": "Bio-Oil Storage",
        "DACCS": "Direct Air Capture with CCS",
        "ERW": "Enhanced Weathering",
        "Macroalgae": "Biomass Sinking",
        "Mineralisation": "Mineral Products",
    },
    inplace=True,
)
df.rename(
    variable=dict([(i, f"Carbon Removal|Novel|{i}") for i in df.variable]),
    inplace=True,
)
df.aggregate(f"Carbon Removal|Novel", append=True)

In [ ]:
df.rename(unit={"t CO2": "t CO2/yr"}, inplace=True)

In [ ]:
df.convert_unit("t CO2/yr", "Mt CO2/yr", inplace=True)

In [ ]:
dsd.validate(df)

In [ ]:
df.set_meta("Chapter 7", "Chapter")

In [ ]:
# df.to_ixmp4("socdr-dev")

In [ ]:
platform = ixmp4.Platform("socdr-dev")

In [ ]:
run = platform.runs.get(**iamc_args)

In [ ]:
run.iamc.remove(run.iamc.tabulate())

In [ ]:
run.iamc.add(df.data)

In [ ]:
platform.units.create("Mt CO2/yr")

In [ ]:
run.meta